In [ ]:
#import functions 

from netops_modules import *

In [ ]:
#read data about devices from the file
FILENAME = 'hosts'

DEVICES = pd.read_excel('xlsx\\{}.xlsx'.format(FILENAME), dtype=str )

#DEVICES['Exec'] = 'X'
#DEVICES['CMD_template'] = 'kibana.template'
#DEVICES['DATAFILE'] = '{}_if.xlsx'.format(FILENAME)

filter = DEVICES['Exec'].notnull() 

DEVICES_RUN=DEVICES[filter]

DEVICES_RUN[HOSTNAME_COLUMN] = DEVICES_RUN[HOSTNAME_COLUMN].replace({pd.np.nan: 'Unknown'}) #change empty HOSTNAME >> 'Unknown'
#DEVICES_RUN[CMDCFG_COLUMN] = None #clear CMGCFG column
DEVICES_RUN.head()

############################################################################################ INVENTORY ##########################################################################################################

In [ ]:
#get running configs and write them into files \\cfg\base\
#function format conn_threads(function, index-column, step(number of threads), DataFrame with device data, attempts=)

conn_threads(get_config_th, 'IP_DEV',100, **DEVICES_RUN, attempts=2)

In [ ]:
#get MAC tables from the devices and parse them
conn_threads(getmacinfo, 'IP_DEV', step = 50, **DEVICES_RUN, attempts=1)

########################################################################################### Information Processing Block ##########################################################################################################

In [ ]:
# Create Excel files from MAC tables

maclist_to_excel(DEVICES_RUN)

In [ ]:
#create dataframe table using data from the files
DEVICES = filedata_to_pd(DEVICES)

# Inventory Parser - model, SN etc..
DEVICES_PARSED = inventory_parser(DEVICES)
#DEVICES_PARSED.head()

DEVICES_RUN = DEVICES_PARSED

DEVICES_RUN.head()

In [ ]:
#write data into xls file without "config" column

DEVICES_PARSED.loc[:,DEVICES_PARSED.columns != 'CFG'].to_excel('xlsx\\{}.xlsx'.format(FILENAME), encoding='cp1251', index=False) #save to ls file

#####################################

####################################

In [ ]:
#find matches MAC >> Interface using Excel tables

get_if(DEVICES_RUN)

####################################

In [ ]:
#find interfaces from configs using "parent" criteria

DEVICES_RUN_IF = find_if(DEVICES_RUN,'^interface')

In [ ]:
#add information about MAC address count on the interfaces

DEVICES_RUN_IF = update_if_info(DEVICES_RUN_IF)

#Save interface information into files

DEVICES_RUN_IF.to_excel('xlsx\\{0}_if.xlsx'.format(FILENAME), encoding='cp1251', index=False) #save to xls file

In [ ]:
#Read Interface data from the file

DEVICES_RUN_IF = pd.read_excel('xlsx\\{0}_if.xlsx'.format(FILENAME), dtype=str)

DEVICES_RUN_IF.head()

In [ ]:
#Save information into files

DEVICES_RUN_IF.to_excel('xlsx\\{0}_if.xlsx'.format(FILENAME), encoding='cp1251', index=False) #save to xls file

DEVICES_RUN_IF_VLA = DEVICES_RUN_IF.drop_duplicates(subset=['IP_DEV','Access VLAN'], keep = 'first')
DEVICES_RUN_IF_VLA = DEVICES_RUN_IF_VLA[DEVICES_RUN_IF_VLA['Access VLAN'].notnull()]
DEVICES_RUN_IF_VLA.to_excel('xlsx\\{0}_vla.xlsx'.format(FILENAME), encoding='cp1251', index=False) #save to xls file

In [ ]:
DEVICES_RUN_IF[DEVICES_RUN['IP_ORIG'] == DEVICES_RUN_IF['IP'] ]

In [ ]:
DEVICES_RUN_IF[DEVICES_RUN_IF['IP_DEV'] == DEVICES_RUN_IF['IP'] or DEVICES_RUN_IF['IP_ORIG'] == DEVICES_RUN_IF['IP'] ].to_excel('xlsx\\{0}_mgmt.xlsx'.format(FILENAME), encoding='cp1251', index=False) #save to xls file

####################################

In [ ]:
#find "strange" ports - min thershold <= IF MAC count < max threshold  

STRANGE_PORTS_PD = find_strange_ports(DEVICES_RUN,2,90,2)

In [ ]:
#Generate commands to send to the devices

DEVICES_RUN = form_cmd_pd(DEVICES_RUN,FILENAME)

########################################################################################## БЛОК Отправка команд на устройства ##########################################################################################################

In [ ]:
# Exec == X - means that command will be send to the device
#send commands to the devices
#function format conn_threads(function, index-column, step(number of threads), DataFrame with device data, attempts=)

conn_threads(sendcmd, 'IP_DEV',50, **DEVICES_RUN)

In [ ]:
#Get results and write them to the file, delete temporary ones

result_to_pd(DEVICES_RUN,FILENAME)

#################DEBUG SECTION###########################